## Building Coding Assistant using Router Chain in LangChain Part 1

In [4]:
%pip install langchain==0.0.252
%pip install openai
%pip install python-dotenv


  Using cached langchain-0.0.252-py3-none-any.whl (1.4 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.263
    Uninstalling langchain-0.0.263:
      Successfully uninstalled langchain-0.0.263
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\forci\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\forci\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\forci\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import openai
import os

load_dotenv()
openai.api_key = os.getenv('openai_api_key')

### Step1: Write prompts for destination Chains 

In [ ]:
python_code_prompt='''You can do coding only and only in python. You are not allowed to code in any other language. \
                      If someone asks you to code in python then do it otherwise answer politely that I am only allowed to code \
                      in python nothing else. Now Code in python for below problem statement delimited by triple ticks\n \
                        ```{input}``` '''


cpp_code_prompt= '''You can do coding only and only in C++. You are not allowed to code in any other language. \
                    If someone asks you to code in C++ then do it otherwise answer politely that I am only allowed to code \
                    in C++ nothing else. Now Code in C++ for below problem statement delimited by triple ticks\n \
                    ```{input}``` '''

sql_code_prompt= '''You can do coding only and only in SQL. You are not allowed to code in any other language. \
                If someone asks you to code in SQL then do it otherwise answer politely that I am only allowed to code \
                in SQL nothing else. Now Code in SQL for below problem statement delimited by triple ticks\n \
                ```{input}``` '''


destinations_chains_info = [
        {
            "name": "Python Expert",
            "description": "This chain is expert to code in python only",
            "prompt_template": python_code_prompt
        },
        {
            "name": "C++ Expert",
            "description": "This chain is expert to code in C++ language only",
            "prompt_template": cpp_code_prompt
        },

        {
            "name": "SQL Expert",
            "description": "This chain is expert to code in SQL language only",
            "prompt_template": sql_code_prompt
        }
    ]


In [ ]:
# llm declaration
llm = OpenAI(temperature=0.00)

### Step2: Declaring the Destination Chains

In [ ]:
destination_chains={}
for each_destination_prompt_info in destinations_chains_info:
    name = each_destination_prompt_info["name"]
    prompt_template = each_destination_prompt_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

print(destination_chains.keys())

default_prompt = PromptTemplate(template="Understand the input and return the code against the below user input. You are only allowed to code nothing else.\n {input}", input_variables=["input"])
default_chain = LLMChain(llm=llm,prompt=default_prompt, output_key="text")

#### Step3: Connect destination chains with router chain

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in destinations_chains_info]
destinations_str = "\n".join(destinations)
print(destinations_str)

In [ ]:
router_prompt= MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

In [ ]:

#router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
        template=router_prompt,
        input_variables=["input"],
        output_parser=RouterOutputParser(),
    )
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(
        router_chain=router_chain,
        destination_chains=destination_chains,
        default_chain=default_chain,
        verbose=True,
    )

language='python'
prefix_prompt=f'''Write code in {language} language. '''

input_prompt='''Take two sorted arrays [3,6,7,9] and [4,5,6,8,10]. Make one sorted array from them'''
final_prompt= prefix_prompt + input_prompt
chain.run({"input": final_prompt, "destinations": destinations_str})

In [ ]:
%pip install black

In [ ]:
import black
# Your Python code in string form
code_string = '\n\n# Code\n\n#Merging two sorted arrays\ndef merge_arrays(arr1, arr2): \n    merged_arr = [] \n    i, j = 0, 0\n    while i < len(arr1) and j < len(arr2): \n        if arr1[i] < arr2[j]: \n            merged_arr.append(arr1[i]) \n            i += 1\n        else: \n            merged_arr.append(arr2[j]) \n            j += 1\n    while i < len(arr1): \n        merged_arr.append(arr1[i]) \n        i += 1\n    while j < len(arr2): \n        merged_arr.append(arr2[j]) \n        j += 1\n    return merged_arr \n  \n# Driver code \narr1 = [3, 6, 7, 9] \narr2 = [4, 5, 6, 8, 10] \n  \nprint(merge_arrays(arr1, arr2))'

# Format the code using black
formatted_code = black.format_str(code_string, mode=black.FileMode())

# Print the formatted code
print(formatted_code)

In [ ]:
language='C++'
prefix_prompt=f'''Write code in {language} language. '''
input_prompt='''Give me the factorial of number 322'''
final_prompt= prefix_prompt + input_prompt
chain.run({"input": final_prompt, "destinations": destinations_str})